![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true)
***
### *Practicum AI:* Transfer - Fruit Classification with Transfer Learning

This exercise was adapted from Baig et al. (2020) <i> The Deep Learning Workshop</i> from <a href="https://www.packtpub.com/product/the-deep-learning-workshop/9781839219856">Packt Publishers</a> (Activity 3.02, page 150).

(10 Minutes)

#### Introduction

In this exercise, we will train a CNN to recognize 120 different kinds of fruit, using the [Fruits 360 dataset](https://arxiv.org/abs/1712.00580).  Horea Muresea and Mihai Oltean first shared this dataset in their article:  *Fruit recognition from images using deep learning, Acta Univ. Sapientiae, Informatica Vol.10, Issue 1, pp.26-42, 2018*.

The Fruits 360 datast contains 90,483 photos of 131 fruits and vegetables.  But rather than use the entire dataset, we use a subset of images - 16,000 photos of 120 different kinds of fruit - in this learning experience.  

#### 1. Import requisite libraries 

In [1]:
import tensorflow as tf

#### 2. Import the dataset and unzip the file.

In [2]:
file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Activity3.02/fruits360.zip'

Download the dataset and save the results to a variable called **zip_dir**.  The `tf.keras.get_file()` function does all this in a single statement.

<div style="padding: 10px;margin-bottom: 20px;border: thin solid #30335D;border-left-width: 10px;background-color: #fff"><strong>Note:</strong> If you downloaded the dataset more than once, subsequent downloads will be slow as the get_file() function updates existing images one at a time.  The function works but is incredibly inefficient.</div>

In [3]:
zip_dir = tf.keras.utils.get_file('fruits360.zip', 
                                   origin       = file_url, 
                                   extract      = True, 
                                   cache_dir    = '/blue/rc-workshops/danielmaxwell',
                                   cache_subdir = 'keras')

82231296/82220233 [==============================] - 2s 0us/step


#### 3. Prepare the data for training

Create a variable named **path** that contains the full path to the fruits360_filtered directory.

In [4]:
import pathlib

In [5]:
path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'

In [6]:
print(path)

/blue/rc-workshops/danielmaxwell/keras/fruits360_filtered


Create the variables **train_dir** and **validation_dir**, with full paths to the train and validation folders.

In [ ]:
train_dir = path / 'Training'
validation_dir = path / 'Test'

Set the number of images for the train and validation datasets to 11398 and 4752 respectively.

```python
total_train = 11398
total_val = 4752
```

In [ ]:
# Code it!

Define two image data generators using class `ImageDataGenerator`, one for train and a second for validation.  Augment the training data by rescaling, rotating, shifting, and flipping the images.  Validation data, though, only needs to be rescaled.

```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale = 1./255, rotation_range = 40, width_shift_range = 0.1, height_shift_range = 0.1, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode = 'nearest')

validation_image_generator = ImageDataGenerator(rescale = 1./255)
```

In [3]:
# Code it!

In [ ]:
# Code it!

In [ ]:
# Code it!

Define four variables (**batch_size**, **img_height**, **img_width**, **channel**) and assign values to each.

```python
batch_size = 16
img_height = 100
img_width  = 100
channel    = 3
```

In [1]:
# Code it!

Define the train and validation data generators using `.flow_from_directory()`.  This step links the two image generators to actual data in a directory.  Also, set the batch_size and target_size arguments.

```python
train_data_gen = train_image_generator.flow_from_directory(batch_size  = batch_size, directory = train_dir, target_size = (img_height, img_width))

val_data_gen = validation_image_generator.flow_from_directory(batch_size  = batch_size, directory = validation_dir, target_size = (img_height, img_width))
```

In [ ]:
# Code it!

In [ ]:
# Code it!

#### 4. Load the pre-trained VGG16 model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

Set seed values to 8 using `np.random_seed()` and `tf.random.set_seed()`.  This allows others to reproduce your results.

```python
np.random.seed(8)
tf.random.set_seed(8)
```

In [ ]:
# Code it!

Load the pre-trained model VGG16 from `tensorflow.keras.applications`.

```python
from tensorflow.keras.applications import VGG16
```

In [35]:
# Code it!

Create a variable called **base_model** and populate it with the following parameters for the VGG16 model.

```python
base_model = VGG16(input_shape = (img_height, img_width, channel), weights = 'imagenet', include_top = False)
```

In [ ]:
# Code it!

### 5. Freeze all the layers in the base VGG16 model

Set the model to non-trainable using the `.trainable` attribute.

```python
base_model.trainable =  False
```

In [4]:
# Code it!

Print a summary of the VGG16 model.

```python
base_model.summary()
```

In [ ]:
# Code it!

We can see that the VGG16 model has 14,714,688 total parameters while the trainable parameters are zero. This is the case because we just froze all the layers of the VGG16 model.

#### 6. Add custom classification layers

Create a new model using `tf.kera.Sequential()` by adding the following layers to the base model:
* A Flatten layer.
* A Dense layer with 1000 units and ReLU activation.
* The final Dense layer with 120 neurons and softmax activation.

```python
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1000, activation = 'relu'),
    layers.Dense(120, activation = 'softmax')
])
```

In [ ]:
# Code it!

#### 7. Compile the model for training

Compile the model with **a)** an Adam optimizer, **b)** a learning rate of 0.001, **c)** the categorical crossentropy loss function, and **d)** an accuracy metric.

```python
optimizer = tf.keras.optimizers.Adam(0.001)

model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

model.summary()
```

In [ ]:
# Code it!

In [ ]:
# Code it!

In [ ]:
# Code it!

#### 8. Train the Model
Train the model using the `.fit()` method.  Provide the train and validation data generators, the steps per epoch, the total train epochs, and the validation steps.

```python
model.fit(
    train_data_gen,
    steps_per_epoch  = total_train // batch_size,
    epochs = 5,
    validation_data  = val_data_gen,
    validation_steps = total_val // batch_size
)
```

In [ ]:
# Code it!

#### Summary
In this notebook, we used transfer learning to customize a pre-trained VGG16 model to classify images from the [Fruits 360 dataset](https://arxiv.org/abs/1712.00580). We replaced the head of the model with our own fully connected layers and then trained it for five epochs.  

The accuracy scores for both training and validation are quite remarkable.  But can you do better?  What might you do differently?